In [1]:
import math
import os.path as op
import mne_bids
from mne.datasets import sample
import data_extracts
from patients_matrix import PatientsMatrix
from coherence_matrix import CoherenceMatrix
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import levene, gaussian_kde
from scipy.stats import anderson
import scipy.stats as stats
import test1
from data_extracts import load_data
from tqdm import tqdm

In [2]:
def show_me_matrix(matrix_flat, name):
    matrix_len = int(math.sqrt(2*len(matrix_flat)+1/4)+1/2)
    matrix = np.zeros(shape=(matrix_len,matrix_len),dtype=float)
    index = 0
    for i in range(matrix_len):
        for j in range(i,matrix_len):
            if i == j:
                matrix[i, j] = 1
            else:
                matrix[i, j] = matrix_flat[index]
                matrix[j, i] = matrix_flat[index]
                index += 1
    plt.imshow(matrix, cmap='viridis')
    plt.title(name)
    plt.colorbar()
    plt.show()
    
    
def creat_matrix(matrix_flat):
    matrix_len = int(math.sqrt(2*len(matrix_flat)+1/4)+1/2)
    matrix = np.zeros(shape=(matrix_len,matrix_len),dtype=float)
    index = 0
    for i in range(matrix_len):
        for j in range(i,matrix_len):
            if i == j:
                matrix[i, j] = 1
            else:
                matrix[i, j] = matrix_flat[index]
                matrix[j, i] = matrix_flat[index]
                index += 1    
    return matrix
    

In [8]:

import keras
from keras import Sequential
from keras import layers

# Define the model
model = Sequential([
  # Input layer for 75x75 grayscale images (modify if your data has color channels)
  layers.Input(shape=(77, 77, 1)),

  # Convolutional layer with 32 filters, 3x3 kernel, ReLU activation
  layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),

  # Max pooling layer with 2x2 pool size
  layers.MaxPooling2D(pool_size=(2, 2)),

  # Convolutional layer with 64 filters, 3x3 kernel, ReLU activation
  layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),

  # Max pooling layer with 2x2 pool size
  layers.MaxPooling2D(pool_size=(2, 2)),

  # Flatten the output of the convolutional layers
  layers.Flatten(),

  # Dense layer with 128 neurons, ReLU activation
  layers.Dense(128, activation="relu"),

  # Dropout layer to prevent overfitting (optional, adjust probability)
  layers.Dropout(0.2),

  # Output layer with 1 neuron for binary classification, sigmoid activation
  layers.Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model (replace with your training data)

model.summary()

In [5]:
# Load the data
from keras.src.legacy.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator object
datagen = ImageDataGenerator(rescale=1./255)

# Load and iterate over the training data
train_data = datagen.flow_from_directory('cnn_test/train', target_size=(77, 77), color_mode='grayscale', class_mode='binary', batch_size=32)
print(type(train_data))
test_data = datagen.flow_from_directory('cnn_test/test', target_size=(77, 77), color_mode='grayscale', class_mode='binary', batch_size=32)

model.fit(train_data, epochs=30)

In [ ]:
# evaluate the model
loss, acc = model.evaluate(test_data)
print("Test accuracy", acc)


In [12]:

from keras import layers, Model

class CustomCNNModel(Model):
    def __init__(self):
        super(CustomCNNModel, self).__init__()
        # Define layers in the constructor
        self.conv1 = layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(77, 77, 1))
        self.pool1 = layers.MaxPooling2D(pool_size=(2, 2))
        self.conv2 = layers.Conv2D(64, kernel_size=(3, 3), activation="relu")
        self.pool2 = layers.MaxPooling2D(pool_size=(2, 2))
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(128, activation="relu")
        self.dropout = layers.Dropout(0.2)
        self.dense2 = layers.Dense(1, activation="sigmoid")
    def call(self, inputs, training=False):
        # Define the forward pass
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        if training:
            x = self.dropout(x, training=training)
        return self.dense2(x)
    
class Trainer:
    def __init__(self, model):
        self.model = model

    def compile(self, loss, optimizer, metrics):
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    def train(self, X_train, y_train, epochs, batch_size, validation_split):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)



In [16]:
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import tensorflow as tf
datagen = ImageDataGenerator(rescale=1./255)
for i in range(45):
    rest_data , film_data = data_extracts.data_extract ('high_gamma','high_gamma',(i,i),data_extracts.data_fit)
    rest_transform = []
    film_transform = []
    for i in range(rest_data.shape[0]):
        rest_transform.append(creat_matrix(rest_data[i]))
        film_transform.append(creat_matrix(film_data[i]))
    
    rest_transform = np.array(rest_transform)
    film_transform = np.array(film_transform)
    
    rest_flow = tf.image.resize(rest_transform, (77,77))
    film_flow = tf.image.resize(film_transform, (77,77))
    
    X_train, X_test, y_train, y_test = test1.data_split(rest_flow, film_flow)
    model = CustomCNNModel()
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.build((32, 77, 77, 1))  # To show the summary, build the model with input shape
    model.summary()
    
    model.fit(X_train, y_train)
    score = model.evaluate(X_test, y_test)
    print(score)
    
    
    
